In [1]:
# WebSocket no Google Colab com ngrok

# Instalar dependências
!pip install pyngrok websockets nest_asyncio

import asyncio
import websockets
import nest_asyncio
from pyngrok import ngrok
import uuid
import json  # Para manipular JSON

# Permitir execução de loops assíncronos no Colab
nest_asyncio.apply()

# Configurar token do ngrok (substitua 'SEU_TOKEN_AQUI' pelo seu token real)
ngrok.set_auth_token("2uTAJZmY4WE0TdhMSjSP6s2zMQB_3J7fXZCSzNJYMxPcbqqKk")

# Lista de conexões ativas
clients = {}

# Criar servidor WebSocket
async def echo(websocket):
    # Atribui um ID único para cada usuário
    user_id = str(uuid.uuid4())  # Cria um ID único para cada cliente
    print(f"Novo cliente conectado: {user_id}")

    # Armazena a conexão do usuário
    clients[user_id] = websocket

    try:
        async for values in websocket:
            print(values)
            if not values.strip():  # Se for vazio ou apenas espaços, ignora
                continue

            try:
                values = json.loads(values)  # Converte a string JSON para dicionário
            except json.JSONDecodeError:
                print(f"Erro ao decodificar JSON: {values}")
                continue  # Pula para a próxima iteração

            user = values.get("user", user_id)  # Se o user não for enviado, usa o ID gerado
            message = values.get("message", "")

            print(f"Recebido de {user}: {message}")

            # Envia a mensagem para todos os clientes conectados
            for client_id, client in clients.items():
                if client != websocket:  # Não envia de volta para quem mandou
                    await client.send(json.dumps({"user": user, "message": message}))
    except websockets.exceptions.ConnectionClosed:
        print(f"Cliente {user_id} desconectado")
    finally:
        # Remove o cliente da lista de conexões quando ele se desconectar
        del clients[user_id]

# Iniciar WebSocket na porta 8765
PORT = 8765
start_server = websockets.serve(echo, "0.0.0.0", PORT)

# Criar um túnel com ngrok
public_url = ngrok.connect(PORT, "http")
print(f"URL do WebSocket: {public_url}")

# Rodar servidor WebSocket
loop = asyncio.get_event_loop()
loop.run_until_complete(start_server)
loop.run_forever()


URL do WebSocket: NgrokTunnel: "https://8793-34-105-27-142.ngrok-free.app" -> "http://localhost:8765"
Novo cliente conectado: 01d5fc35-894b-41ef-8588-deb73db3ad8f
Novo cliente conectado: 467a4319-0d21-4223-9379-9cb24eca1fce
{"user":"midoria","message":"Olá Jony!"}
Recebido de midoria: Olá Jony!
{"user":"midoria","message":"Olá Midoria!"}
Recebido de midoria: Olá Midoria!


KeyboardInterrupt: 